# Обработка первого файла 'casc-resto.csv'

### Чтение файла

In [175]:
import pandas as pd

In [176]:
df1 = pd.read_csv('casc-resto.csv', sep=';', decimal=',')

In [177]:
df1.head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2.0,2.0
1,2903215,43,2015-04-07,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
2,2748887,43,2015-05-22,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
3,2862077,46,2015-03-05,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
4,2862077,46,2015-03-11,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0


In [178]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882222 entries, 0 to 882221
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   CustomerID            882222 non-null  int64  
 1   Restaurant            882222 non-null  int64  
 2   RKDate                882222 non-null  object 
 3   RegionName            882222 non-null  object 
 4   BrandsNames           882222 non-null  object 
 5   DishCategoryName      881608 non-null  object 
 6   Quantity              882222 non-null  int64  
 7   SummBasic             882222 non-null  float64
 8   SummAfterPointsUsage  882222 non-null  float64
dtypes: float64(2), int64(3), object(4)
memory usage: 60.6+ MB


### Приведение RKDate к datetime

In [179]:
df1['RKDate'] = pd.to_datetime(df1['RKDate'])

In [180]:
df1.describe()

,CustomerID,Restaurant,Quantity,SummBasic,SummAfterPointsUsage
count,8.822220e+05,882222.000000,882222.000000,882222.000000,882222.000000
mean,2.809199e+06,468.268804,1.154900,287.725795,260.255589
std,6.178420e+04,279.170967,0.650287,264.555291,251.289517
min,2.728046e+06,40.000000,0.000000,0.500000,-2593.000000
25%,2.754886e+06,333.000000,1.000000,120.000000,110.000000
50%,2.785104e+06,434.000000,1.000000,225.000000,199.000000
75%,2.878900e+06,712.000000,1.000000,379.000000,348.940000
max,2.913132e+06,980.000000,107.000000,29450.000000,29450.000000


### Удаление подозрительных строк (SummAfterPointsUsage как отриц. число)

In [181]:
df1 = df1[df1['SummAfterPointsUsage']>0]

In [182]:
df1.describe()

,CustomerID,Restaurant,Quantity,SummBasic,SummAfterPointsUsage
count,8.822210e+05,882221.000000,882221.000000,882221.000000,882221.000000
mean,2.809199e+06,468.269228,1.154899,287.724840,260.258823
std,6.178419e+04,279.170841,0.650287,264.553921,251.271298
min,2.728046e+06,40.000000,0.000000,0.500000,0.250000
25%,2.754886e+06,333.000000,1.000000,120.000000,110.000000
50%,2.785104e+06,434.000000,1.000000,225.000000,199.000000
75%,2.878900e+06,712.000000,1.000000,379.000000,348.940000
max,2.913132e+06,980.000000,107.000000,29450.000000,29450.000000


## Группировка по покупателям

In [183]:
df_clients = df1.groupby(df1['CustomerID'])

### Получение T/F, тех кто был в ресторанах с 2017-07-01 по 2017-12-31

In [184]:
df_clients2 = df_clients.apply(lambda x: any((x["RKDate"] >'2017-07-01') & (x['RKDate'] < '2017-12-31')))
#result - определение тех, кто был в те даты в ресторанах

### Получение Recency -> (2017-07-01) - (последний приход посетителя до 2017-07-01)
Если прихода не было, то определяем как null

In [185]:
import numpy as np
frompoint = pd.Timestamp('2017-07-01')
#recency = df_clients.apply(lambda x: frompoint - max(x["RKDate"] <'2017-07-01')))
recency = df_clients.apply(lambda x: frompoint - max(x[x["RKDate"]<frompoint]["RKDate"]) if (len(x[x["RKDate"]<frompoint])>0) else np.nan)

### Получение Frequency = (Сумма всех посещений до 2017-07-01) / (Разность между 1 датой и 2017-07-01 в месяцах)

In [186]:
frequency = df_clients.apply(lambda x: len(x[x["RKDate"]<frompoint]["RKDate"]) / ((frompoint - min(x["RKDate"]))/np.timedelta64(1, 'M')) if ((frompoint - min(x["RKDate"]))/np.timedelta64(1, 'M'))>0 else 0)

### Получение Monetary_Value - средний чек клиента до 2017-07-01

In [187]:
monetary_value = df_clients.apply(lambda x: sum(x[x["RKDate"]<frompoint]["SummBasic"]) / len(x[x["RKDate"]<frompoint]) if len(x[x["RKDate"]<frompoint])>0 else 0)

### Пусть еще одним параметром будет стандартное отклонение (stdev) суммы заказа до 2017-07-01
Если заказ был только 1, то stdev -> 0

In [188]:
import math
std_summ = df_clients.apply(lambda x: math.sqrt(sum(((x[x["RKDate"]<frompoint]["SummBasic"] - (sum(x[x["RKDate"]<frompoint]["SummBasic"]) / len(x[x["RKDate"]<frompoint])) if len(x[x["RKDate"]<frompoint])>0 else 0))**2) / (len(x[x["RKDate"]<frompoint]) - 1)) if (len(x[x["RKDate"]<frompoint]) - 1 > 0) else 0)

In [189]:
print(std_summ)

CustomerID
2728046    203.957054
2728088    144.555511
2728089    209.018458
2728095    173.963685
2728107    277.736806
              ...    
2913103    442.595405
2913106    188.197483
2913114    248.340631
2913123    126.392808
2913132    150.080588
Length: 10000, dtype: float64


### Recency пока что определяется как timedelta, а не число, поэтому приводим к числу:

In [190]:
recencyMonths = recency / np.timedelta64(1, 'D')

In [191]:
recencyMonths

CustomerID
2728046    160.0
2728088    720.0
2728089     15.0
2728095    177.0
2728107    115.0
           ...  
2913103    401.0
2913106    126.0
2913114     95.0
2913123      8.0
2913132     44.0
Length: 10000, dtype: float64

### Строим таблицу из задания 2 - [Y, Recency, Frequency, Monetary_value, Std_sum]

In [192]:
try_dataframe = pd.DataFrame(data = {'Y': df_clients2,'Recency': recencyMonths,'Frequency': frequency,'Monetary_value': monetary_value,'Std_sum': std_summ})

In [193]:
try_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2728046 to 2913132
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Y               10000 non-null  bool   
 1   Recency         9938 non-null   float64
 2   Frequency       10000 non-null  float64
 3   Monetary_value  10000 non-null  float64
 4   Std_sum         10000 non-null  float64
dtypes: bool(1), float64(4)
memory usage: 400.4 KB


### Сохраняем временную таблицу

In [194]:
try_dataframe.to_csv(r'Temporary1.csv')

# Обработка второго файла 'CASC_Constant.csv'

In [195]:
df2=pd.read_csv('CASC_Constant.csv')

In [196]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   ActivationDate   10000 non-null  object 
 2   Age              9953 non-null   float64
 3   Sex              8874 non-null   object 
 4   SubscribedEmail  10000 non-null  bool   
 5   SubscribedPush   10000 non-null  bool   
dtypes: bool(2), float64(1), int64(1), object(2)
memory usage: 332.2+ KB


### Заменяем пустые и экстремальные значения Возраста на медиану

In [197]:
meanAge = df2['Age'].median()

In [198]:
df2.loc[df2.Age > 99, 'Age'] = meanAge

In [199]:
df2['Age'] = df2['Age'].fillna(meanAge)

In [200]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   ActivationDate   10000 non-null  object 
 2   Age              10000 non-null  float64
 3   Sex              8874 non-null   object 
 4   SubscribedEmail  10000 non-null  bool   
 5   SubscribedPush   10000 non-null  bool   
dtypes: bool(2), float64(1), int64(1), object(2)
memory usage: 332.2+ KB


### Приводим Пол к категориальному параметру

In [201]:
df2['Sex'].value_counts()

Female    5141
Male      3733
Name: Sex, dtype: int64

In [202]:
df2['Sex'] = df2['Sex'].fillna('Not_chosen')

In [203]:
df2['Sex'] = df2['Sex'].astype('category').cat.codes

In [204]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   ActivationDate   10000 non-null  object 
 2   Age              10000 non-null  float64
 3   Sex              10000 non-null  int8   
 4   SubscribedEmail  10000 non-null  bool   
 5   SubscribedPush   10000 non-null  bool   
dtypes: bool(2), float64(1), int64(1), int8(1), object(1)
memory usage: 263.8+ KB


In [205]:
df2['Sex'].value_counts()

0    5141
1    3733
2    1126
Name: Sex, dtype: int64

### Удаляем ненужные столбцы, а також определяем CustomerId как индекс для Df2

In [206]:
df2 = df2.drop('ActivationDate', 1)

In [207]:
df2 = df2.drop(['SubscribedEmail', 'SubscribedPush'], 1)

In [208]:
df2=df2.set_index('CustomerId')

In [209]:
df2

,Age,Sex
CustomerId,,
2728183,21.0,0
2728198,21.0,0
2728306,21.0,0
2728178,22.0,1
2728322,22.0,1
...,...,...
2912655,49.0,0
2912502,52.0,1
2912767,52.0,1


## Связываем первую таблицу со второй

In [210]:
final_table = try_dataframe.join(df2)

In [211]:
final_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 2728046 to 2913132
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Y               10000 non-null  bool   
 1   Recency         9938 non-null   float64
 2   Frequency       10000 non-null  float64
 3   Monetary_value  10000 non-null  float64
 4   Std_sum         10000 non-null  float64
 5   Age             10000 non-null  float64
 6   Sex             10000 non-null  int8   
dtypes: bool(1), float64(5), int8(1)
memory usage: 808.3 KB


In [212]:
final_table.head()

,Y,Recency,Frequency,Monetary_value,Std_sum,Age,Sex
CustomerID,,,,,,,
2728046,False,160.0,1.902305,254.807018,203.957054,24.0,0
2728088,False,720.0,1.134708,279.647059,144.555511,46.0,1
2728089,True,15.0,1.101334,422.181818,209.018458,27.0,0
2728095,False,177.0,3.304003,289.676768,173.963685,54.0,0
2728107,True,115.0,0.700849,414.333333,277.736806,48.0,1


### Выбрасываем значения, если клиент не приходил ни разу до 2017-01-01

In [213]:
withRecencyDF = final_table.drop(final_table[final_table['Recency'].isnull()].index)

In [214]:
withRecencyDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9938 entries, 2728046 to 2913132
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Y               9938 non-null   bool   
 1   Recency         9938 non-null   float64
 2   Frequency       9938 non-null   float64
 3   Monetary_value  9938 non-null   float64
 4   Std_sum         9938 non-null   float64
 5   Age             9938 non-null   float64
 6   Sex             9938 non-null   int8   
dtypes: bool(1), float64(5), int8(1)
memory usage: 485.3 KB


### Y_table - таблица зависимой переменной Y

In [215]:
Y_table = withRecencyDF.Y

In [216]:
Y_table

CustomerID
2728046    False
2728088    False
2728089     True
2728095    False
2728107     True
           ...  
2913103    False
2913106    False
2913114    False
2913123     True
2913132     True
Name: Y, Length: 9938, dtype: bool

In [217]:
from sklearn.model_selection import train_test_split

### Удаляем столбец Y из таблицы параметров

In [218]:
withRecencyDF.drop('Y', 1, inplace = True)

In [219]:
withRecencyDF

,Recency,Frequency,Monetary_value,Std_sum,Age,Sex
CustomerID,,,,,,
2728046,160.0,1.902305,254.807018,203.957054,24.0,0
2728088,720.0,1.134708,279.647059,144.555511,46.0,1
2728089,15.0,1.101334,422.181818,209.018458,27.0,0
2728095,177.0,3.304003,289.676768,173.963685,54.0,0
2728107,115.0,0.700849,414.333333,277.736806,48.0,1
...,...,...,...,...,...,...
2913103,401.0,1.442331,450.307692,442.595405,32.0,2
2913106,126.0,6.915791,282.711230,188.197483,23.0,1
2913114,95.0,4.105095,345.873874,248.340631,27.0,0


### Деление на обучающую и тестовую выборки

In [220]:
X_train, X_test, y_train, y_test = train_test_split(withRecencyDF, Y_table, test_size=0.2, random_state=42)

In [221]:
from sklearn.linear_model import LogisticRegression

### Построение логистической регрессии

In [222]:
reg_all = LogisticRegression()

In [223]:
reg_all.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [224]:
y_pred = reg_all.predict(X_test)

### Подсчет параметров-метрик для оценки модели

In [225]:
reg_all.score(X_test, y_test)

0.7344064386317908

In [226]:
from sklearn.metrics import precision_recall_fscore_support

In [227]:
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.7370405880472994, 0.738124868139025, 0.7343287299278053, None)

In [228]:
from sklearn.metrics import classification_report

In [229]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.79      0.69      0.74      1077
        True       0.68      0.78      0.73       911

    accuracy                           0.73      1988
   macro avg       0.74      0.74      0.73      1988
weighted avg       0.74      0.73      0.73      1988

